In [3]:
import os
import getpass
import openai
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Milvus
from langchain.document_loaders import TextLoader
from langchain.document_loaders import Docx2txtLoader
openai_api_key = os.getenv("OPENAI_API_KEY")

loader = Docx2txtLoader("data/NEC_core_clause.docx")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)


In [4]:
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [8]:
vector_db = Milvus.from_documents(
    docs,
    embeddings,
    connection_args={"host": "127.0.0.1", "port": "19530"},
)
query = "When should the Employer make the payment?"
docs = vector_db.similarity_search(query)
docs[0].page_content

'four weeks after the Supervisor issues the Defects Certificate or\n\nthirteen weeks after the Project Manager issues a termination certificate.\n\nThe Project Manager gives the Contractor details of how the amount due has been assessed. The final payment is made within three weeks of the assessment or, if a different period is stated in the Contract Data, within the period stated.\n\n\tIf the Project Manager does not make this assessment within the time allowed, the Contractor may issue to the Client an assessment of the final amount due, giving details of how the final amount due has been assessed. If the Client agrees with this assessment, a final payment is made within three weeks of the assessment or, if a different period is stated in the Contract Data, within the period stated.'

'Subject to clause \u200e20.12, the Owner shall pay to the Onshore Contractor the sum certified as due to the Onshore Contractor in an Interim Certificate within twenty (20) Days after the date of issue thereof provided that there is no Defect in any portion of the Works in respect of which such payment is proposed and the Onshore Contractor can demonstrate functional and consistent compliance with Environmental and Safety Laws and the Project’s HSE Management Plan. If there is any Defect in any portion of the Works in respect of which payment in proposed by the Onshore Contractor and such Defect is not material as determined by the Owner in its sole discretion, the Owner shall be entitled to make payment of the sum certified as due to the Onshore Contractor less any amount that the Owner considers should be deducted in respect of the Defect.\n\nFinal Completion Certificate'

In [9]:
query = "When should the Employer issue the interim payment certificate?"
docs = vector_db.similarity_search(query)
docs[0].page_content

'only be obligated to issue the Interim Certificate in relation to such application for the issue of an Interim Certificate in accordance with the terms and conditions set forth below including but not limited to in Clause 20.5 (Owner to Review the Application for Payment); or\n\nshall under no circumstances make application for the issue of an Interim Certificate in respect of Milestone Events if the Onshore Contractor is in breach of any of its bonding obligations under Clause 20 (LNTP Bond, Advance Payment Bond, Performance Bond, and Legal Opinion).\n\nOwner to Review the Application for Payment\n\nOn the Owner’s receipt of an application from the Onshore Contractor that complies with the requirements of clause \u200e20.4, the Owner shall issue the Interim Certificate within fourteen 14 Days, provided that:'

'only be obligated to issue the Interim Certificate in relation to such application for the issue of an Interim Certificate in accordance with the terms and conditions set forth below including but not limited to in Clause 20.5 (Owner to Review the Application for Payment); or\n\nshall under no circumstances make application for the issue of an Interim Certificate in respect of Milestone Events if the Onshore Contractor is in breach of any of its bonding obligations under Clause 20 (LNTP Bond, Advance Payment Bond, Performance Bond, and Legal Opinion).\n\nOwner to Review the Application for Payment\n\nOn the Owner’s receipt of an application from the Onshore Contractor that complies with the requirements of clause \u200e20.4, the Owner shall issue the Interim Certificate within fourteen 14 Days, provided that:'


'only be obligated to issue the Interim Certificate in relation to such application for the issue of an Interim Certificate in accordance with the terms and conditions set forth below including but not limited to in Clause 20.5 (Owner to Review the Application for Payment); or\n\nshall under no circumstances make application for the issue of an Interim Certificate in respect of Milestone Events if the Onshore Contractor is in breach of any of its bonding obligations under Clause 20 (LNTP Bond, Advance Payment Bond, Performance Bond, and Legal Opinion).\n\nOwner to Review the Application for Payment\n\nOn the Owner’s receipt of an application from the Onshore Contractor that complies with the requirements of clause \u200e20.4, the Owner shall issue the Interim Certificate within fourteen 14 Days, provided that:'

In [4]:
def gpt_response(prompt, temp=0.1):
    primer = f"""You are a constuction contract expert. Here are some knowledge you should remember: The payment process in a construction project often includes these important events placed in time order: Employer receives Milestone Payment Application, Employer notifies Contractor about the satisfaction of the Milestone, Employer issues Milestone Completion Certificate, Employer makes Milestone Payment. Notice that there is a fine line between "Employer notifies Contractor about the satisfaction of the Milestone" and "Employer issues Milestone Completion Certificate": when Employer make clear that the application is approved or certify relevant documents (certificate), consider this event as "Employer issues Milestone Completion Certificate", elsewise the Employer still hold the possibility of further revise or reject, it is considered as "Employer notifies Contractor about the satisfaction of the Milestone".\
    Also, many concepts have multiple expressions. Supplier also means Contractor, Employer sometimes is called Purchaser, Owner, or the Company, it can also be represented by Employer's Engineer. Employer receives can be considered happen simultaneously as Contractor submits. Payment application, also known as payment request or payment certificate, is the statement with supporting documents (e.g. project report) submitted to the employer showing the amount that the Contractor considers to be due. Practical completion is sometimes described as being the point at which a building is complete.\
"""
    res = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[
            {"role": "system", "content": primer},
            {"role": "user", "content": prompt}
        ],
        temperature = temp,
    )

    return res['choices'][0]['message']['content']

In [11]:
res = gpt_response(joint_extraction_prompt)
print(res)

["28 days", "21 days", "56 days", "Not mentioned"]
